<a href="https://colab.research.google.com/github/dimna21/ML_Final_Project/blob/main/model_experiment_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install darts
!pip install mlflow
!pip install dagshub

In [4]:
import mlflow
import dagshub
import json

In [50]:
dagshub.init(repo_owner='dimna21', repo_name='ML_Final_Project', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=6f43284d-9b79-46d6-8841-440866d492bf&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=1c3b15238da61499d6b1c550962a11819ac1c23e7d02a4ddbbbcb9012460b422




Accessing as dimna21

Repository ML_Final_Project doesn't exist, creating it under current user.

Initialized MLflow to track repo "dimna21/ML_Final_Project"

Repository dimna21/ML_Final_Project initialized!

#No covariates approach

In [14]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/test.csv')
features = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/features.csv')
stores = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/stores.csv')

In [70]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from darts import TimeSeries
from darts.models.forecasting.arima import ARIMA
import pickle

class DataCompleter(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.all_dates = None
        self.all_stores = None
        self.all_depts = None
        self.actual_combinations = None

    def fit(self, X, y=None):
        self.all_dates = X['Date'].unique()
        self.all_stores = X['Store'].unique()
        self.all_depts = X['Dept'].unique()
        self.actual_combinations = set(X.groupby(['Store', 'Dept']).size().index.tolist())
        return self

    def transform(self, X):
        from itertools import product

        complete_combinations = []
        for date in self.all_dates:
            for store, dept in self.actual_combinations:
                complete_combinations.append((date, store, dept))

        complete_df = pd.DataFrame(complete_combinations, columns=['Date', 'Store', 'Dept'])

        result = complete_df.merge(X, on=['Date', 'Store', 'Dept'], how='left')
        result['Weekly_Sales'] = result['Weekly_Sales'].fillna(0)
        result['IsHoliday'] = result['IsHoliday'].fillna(False)

        return result

class DepartmentProportionLearner(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.dept_proportions = None

    def fit(self, X, y=None):
        proportions_list = []

        for store in X['Store'].unique():
            store_data = X[X['Store'] == store]
            store_totals = store_data.groupby('Date')['Weekly_Sales'].sum()

            for dept in store_data['Dept'].unique():
                dept_data = store_data[store_data['Dept'] == dept]
                dept_with_totals = dept_data.merge(
                    store_totals.rename('Store_Total').reset_index(), on='Date'
                )
                dept_with_totals['Proportion'] = dept_with_totals['Weekly_Sales'] / dept_with_totals['Store_Total']
                dept_with_totals['Proportion'] = dept_with_totals['Proportion'].fillna(0)

                proportions_list.append({
                    'Store': store,
                    'Dept': dept,
                    'Avg_Proportion': dept_with_totals['Proportion'].mean()
                })

        self.dept_proportions = pd.DataFrame(proportions_list)
        return self

    def transform(self, X):
        return X

class StoreAggregator(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        aggregated = X.groupby(['Date', 'Store']).agg({
            'Weekly_Sales': 'sum',
            'IsHoliday': 'first'
        }).reset_index()
        return aggregated

class StoreARIMATrainer(BaseEstimator, TransformerMixin):

    def __init__(self, p=1, d=1, q=1):
        self.p = p
        self.d = d
        self.q = q
        self.store_models = {}
        self.trained_stores = []

    def fit(self, X, y=None):
        self.store_models = {}
        self.trained_stores = []

        for store in X['Store'].unique():
            try:
                store_data = X[X['Store'] == store].copy()
                store_data = store_data.sort_values('Date').reset_index(drop=True)

                if len(store_data) < 10:
                    continue

                store_ts = TimeSeries.from_dataframe(
                    store_data, time_col='Date', value_cols=['Weekly_Sales'],
                    fill_missing_dates=False, freq=None
                )

                model = ARIMA(p=self.p, d=self.d, q=self.q)
                model.fit(store_ts)

                self.store_models[store] = model
                self.trained_stores.append(store)

            except Exception as e:
                print(f"Failed to train store {store}: {str(e)}")
                continue

        print(f"Successfully trained {len(self.store_models)} store models")
        return self

    def transform(self, X):
        return X

class StorePredictor(BaseEstimator, TransformerMixin):

    def __init__(self, prediction_periods):
        self.prediction_periods = prediction_periods
        self.store_models = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.store_models is None:
            raise ValueError("Store models not available. Ensure StoreARIMATrainer was fitted first.")

        predictions_list = []

        for store in self.store_models.keys():
            try:
                model = self.store_models[store]
                pred = model.predict(self.prediction_periods)

                pred_values = pred.values().flatten()
                pred_dates = pred.time_index

                for i, date in enumerate(pred_dates):
                    predictions_list.append({
                        'Store': store,
                        'Date': date,
                        'Weekly_Sales': pred_values[i]
                    })

            except Exception as e:
                print(f"Failed to predict for store {store}: {str(e)}")
                continue

        return pd.DataFrame(predictions_list)

class DepartmentDisaggregator(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.dept_proportions = None

    def fit(self, X, y=None):
        # Will be set by the pipeline
        return self

    def transform(self, X):
        if self.dept_proportions is None:
            raise ValueError("Department proportions not available.")

        dept_predictions_list = []

        for _, row in X.iterrows():
            store = row['Store']
            date = row['Date']
            store_sales = row['Weekly_Sales']

            store_depts = self.dept_proportions[self.dept_proportions['Store'] == store]

            for _, dept_row in store_depts.iterrows():
                dept = dept_row['Dept']
                proportion = dept_row['Avg_Proportion']
                dept_sales = store_sales * proportion

                dept_predictions_list.append({
                    'Store': store,
                    'Dept': dept,
                    'Date': date,
                    'Weekly_Sales': dept_sales
                })

        return pd.DataFrame(dept_predictions_list)

class SubmissionFormatter(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['Date'] = pd.to_datetime(X['Date'])
        X['Date_str'] = X['Date'].dt.strftime('%Y-%m-%d')
        X['Id'] = (X['Store'].astype(int).astype(str) + '_' +
                  X['Dept'].astype(int).astype(str) + '_' +
                  X['Date_str'])
        return X[['Id', 'Weekly_Sales']]

class PipelineConnector(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.dept_proportions = None
        self.store_models = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X

class HierarchicalARIMAPipeline:
    """Custom pipeline that connects the transformers properly"""

    def __init__(self, prediction_periods, p=1, d=1, q=1):
        self.prediction_periods = prediction_periods

        # Initialize transformers
        self.data_completer = DataCompleter()
        self.dept_learner = DepartmentProportionLearner()
        self.store_aggregator = StoreAggregator()
        self.arima_trainer = StoreARIMATrainer(p=p, d=d, q=q)
        self.store_predictor = StorePredictor(prediction_periods=prediction_periods)
        self.dept_disaggregator = DepartmentDisaggregator()
        self.formatter = SubmissionFormatter()

        self.fitted = False

    def fit(self, X, y=None):
        print("=== FITTING HIERARCHICAL ARIMA PIPELINE ===")

        # Step 1: Complete data
        X1 = self.data_completer.fit_transform(X)

        # Step 2: Learn department proportions
        X2 = self.dept_learner.fit_transform(X1)

        # Step 3: Aggregate to store level
        X3 = self.store_aggregator.fit_transform(X2)

        # Step 4: Train ARIMA models
        X4 = self.arima_trainer.fit_transform(X3)

        # Connect learned parameters
        self.store_predictor.store_models = self.arima_trainer.store_models
        self.dept_disaggregator.dept_proportions = self.dept_learner.dept_proportions

        self.fitted = True
        print("✅ Pipeline fitted successfully!")
        return self

    def predict(self, X=None):
        if not self.fitted:
            raise ValueError("Pipeline must be fitted first")

        print(f"=== MAKING PREDICTIONS ===")

        # Step 5: Make store predictions
        store_preds = self.store_predictor.transform(None)

        # Step 6: Disaggregate to departments
        dept_preds = self.dept_disaggregator.transform(store_preds)

        # Step 7: Format for submission
        submission = self.formatter.transform(dept_preds)

        return submission

    def predict_for_test(self, test_data, save_path=None):
        """Make predictions that exactly match test data requirements"""
        if not self.fitted:
            raise ValueError("Pipeline must be fitted first")

        print(f"=== MAKING TEST PREDICTIONS ===")
        print(f"Test data shape: {test_data.shape}")

        # Make predictions
        dept_preds = self.predict()

        # Ensure proper data types for merging
        test_copy = test_data.copy()
        test_copy['Date'] = pd.to_datetime(test_copy['Date'])
        test_copy['Store'] = test_copy['Store'].astype(int)
        test_copy['Dept'] = test_copy['Dept'].astype(int)

        # Create test Id format
        test_copy['Date_str'] = test_copy['Date'].dt.strftime('%Y-%m-%d')
        test_copy['Id'] = (test_copy['Store'].astype(str) + '_' +
                          test_copy['Dept'].astype(str) + '_' +
                          test_copy['Date_str'])

        # Ensure predictions have proper data types
        dept_preds_copy = dept_preds.copy()
        # Extract Store, Dept, Date from Id for merging
        id_parts = dept_preds_copy['Id'].str.split('_', expand=True)
        dept_preds_copy['Store'] = id_parts[0].astype(int)
        dept_preds_copy['Dept'] = id_parts[1].astype(int)
        dept_preds_copy['Date'] = pd.to_datetime(id_parts[2])

        # Merge test requirements with predictions
        test_with_preds = test_copy[['Id', 'Store', 'Dept', 'Date']].merge(
            dept_preds_copy[['Store', 'Dept', 'Date', 'Weekly_Sales']],
            on=['Store', 'Dept', 'Date'],
            how='left'
        )

        # Fill missing predictions with 0
        test_with_preds['Weekly_Sales'] = test_with_preds['Weekly_Sales'].fillna(0)

        # Create final submission
        final_submission = test_with_preds[['Id', 'Weekly_Sales']].copy()

        print(f"Final submission shape: {final_submission.shape}")
        print(f"Required shape: ({len(test_data)}, 2)")
        print(f"Shape matches: {final_submission.shape == (len(test_data), 2)}")
        print(f"Missing predictions filled: {test_with_preds['Weekly_Sales'].isna().sum()}")

        # Save if path provided
        if save_path:
            final_submission.to_csv(save_path, index=False)
            print(f"✅ Submission saved to: {save_path}")

        return final_submission

    def validate_parameters(self, train_data, validation_weeks=12, param_list=None):
        """
        Validate different ARIMA parameters using train/validation split with MLflow logging
        """
        import mlflow
        import json

        if param_list is None:
            param_list = [(1,1,1), (2,1,1), (1,1,2), (2,1,2), (1,0,1)]

        try:
            mlflow.end_run()
        except:
            pass

        mlflow.set_experiment("parameter_validation")

        with mlflow.start_run(run_name="arima_parameter_validation"):
            print(f"=== PARAMETER VALIDATION ===")
            print(f"Testing {len(param_list)} parameter combinations")
            print(f"Validation weeks: {validation_weeks}")

            # Log validation setup
            mlflow.log_param("validation_weeks", validation_weeks)
            mlflow.log_param("param_combinations", len(param_list))
            mlflow.log_param("param_list", str(param_list))

            # Convert dates and sort
            train_data = train_data.copy()
            train_data['Date'] = pd.to_datetime(train_data['Date'])
            train_data = train_data.sort_values('Date')

            # Find split point (last validation_weeks of data)
            unique_dates = sorted(train_data['Date'].unique())
            split_date = unique_dates[-(validation_weeks + 1)]

            train_split = train_data[train_data['Date'] <= split_date].copy()
            val_split = train_data[train_data['Date'] > split_date].copy()

            # Log split information
            mlflow.log_param("train_split_size", len(train_split))
            mlflow.log_param("val_split_size", len(val_split))
            mlflow.log_param("split_date", str(split_date))

            results = []
            validation_scores = {}

            for i, (p, d, q) in enumerate(param_list):
                print(f"\n--- Testing ARIMA({p},{d},{q}) ({i+1}/{len(param_list)}) ---")

                # Create nested run for each parameter combination
                with mlflow.start_run(run_name=f"ARIMA({p},{d},{q})", nested=True):
                    try:
                        # Log parameters for this combination
                        mlflow.log_param("p", p)
                        mlflow.log_param("d", d)
                        mlflow.log_param("q", q)

                        # Create pipeline with current parameters (without MLflow to avoid conflicts)
                        pipeline = HierarchicalARIMAPipeline(
                            prediction_periods=validation_weeks,
                            p=p, d=d, q=q
                        )

                        # Fit on train split
                        pipeline.fit(train_split)

                        # Make predictions for validation period
                        val_predictions = pipeline.predict()

                        # Calculate validation error
                        error_score = self._calculate_validation_error(val_split, val_predictions)

                        # Log results
                        mlflow.log_metric("wmae", error_score)
                        mlflow.log_param("status", "success")

                        results.append({
                            'params': (p, d, q),
                            'error': error_score,
                            'status': 'success',
                            'pipeline': pipeline
                        })

                        validation_scores[f"ARIMA({p},{d},{q})"] = error_score
                        print(f"✅ ARIMA({p},{d},{q}) - WMAE: {error_score:.2f}")

                    except Exception as e:
                        # Log failure
                        mlflow.log_param("status", "failed")
                        mlflow.log_param("error_message", str(e))
                        mlflow.log_metric("wmae", float('inf'))

                        print(f"❌ ARIMA({p},{d},{q}) - Failed: {str(e)}")
                        results.append({
                            'params': (p, d, q),
                            'error': float('inf'),
                            'status': 'failed',
                            'error_msg': str(e)
                        })

                        validation_scores[f"ARIMA({p},{d},{q})"] = float('inf')

            # Sort by error (best first)
            successful_results = [r for r in results]
            successful_results.sort(key=lambda x: x['error'])

            # Log overall results
            try:
                with open("validation_results.json", "w") as f:
                    json.dump(validation_scores, f, indent=2)
                mlflow.log_artifact("validation_results.json")
            except Exception as e:
                print(f"Warning: Could not save validation results: {e}")

            if successful_results:
                print("Best parameters (by WMAE):")

                # Log best results
                best_result = successful_results[0]
                mlflow.log_metric("best_wmae", best_result['error'])
                mlflow.log_param("best_params", str(best_result['params']))
                mlflow.log_metric("successful_combinations", len(successful_results))
                mlflow.log_metric("failed_combinations", len(param_list) - len(successful_results))

                for i, result in enumerate(successful_results[:3]):
                    p, d, q = result['params']
                    print(f"{i+1}. ARIMA({p},{d},{q}) - WMAE: {result['error']:.2f}")
                    mlflow.log_metric(f"rank_{i+1}_wmae", result['error'])
                    mlflow.log_param(f"rank_{i+1}_params", f"({p},{d},{q})")

                # Return best pipeline
                print(f"\n✅ Best parameters: ARIMA{best_result['params']} with WMAE: {best_result['error']:.2f}")
                return best_result['pipeline'], results
            else:
                mlflow.log_param("validation_status", "all_failed")
                mlflow.log_metric("successful_combinations", 0)
                mlflow.log_metric("failed_combinations", len(param_list))

                print("❌ No successful parameter combinations")
                return None, results

    def _calculate_validation_error(self, val_actual, val_predictions):
        """Calculate WMAE (Weighted Mean Absolute Error) for validation"""
        try:
            # Prepare actual data
            val_actual = val_actual.copy()
            val_actual['Date'] = pd.to_datetime(val_actual['Date'])

            # Prepare predictions data
            val_predictions = val_predictions.copy()
            if 'Id' in val_predictions.columns:
                # Extract components from Id
                id_parts = val_predictions['Id'].str.split('_', expand=True)
                val_predictions['Store'] = id_parts[0].astype(int)
                val_predictions['Dept'] = id_parts[1].astype(int)
                val_predictions['Date'] = pd.to_datetime(id_parts[2])
            else:
                val_predictions['Date'] = pd.to_datetime(val_predictions['Date'])
                val_predictions['Store'] = val_predictions['Store'].astype(int)
                val_predictions['Dept'] = val_predictions['Dept'].astype(int)

            # Merge actual and predicted
            comparison = val_actual.merge(
                val_predictions[['Store', 'Dept', 'Date', 'Weekly_Sales']],
                on=['Store', 'Dept', 'Date'],
                how='inner',
                suffixes=('_actual', '_pred')
            )

            if len(comparison) == 0:
                print("Warning: No matching predictions for validation")
                return float('inf')

            # Calculate WMAE (Weighted Mean Absolute Error)
            # This is the Kaggle competition metric
            weights = comparison['Weekly_Sales_actual'].abs()
            errors = (comparison['Weekly_Sales_actual'] - comparison['Weekly_Sales_pred']).abs()

            if weights.sum() == 0:
                return float('inf')

            wmae = (weights * errors).sum() / weights.sum()

            print(f"Validation pairs matched: {len(comparison)}")
            return wmae

        except Exception as e:
            print(f"Error calculating validation metric: {str(e)}")
            return float('inf')



In [71]:
# Load data
train = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/test.csv')

# Test different ARIMA parameters with validation
pipeline = HierarchicalARIMAPipeline(prediction_periods=39) # Don't specify p,d,q yet

# Find best parameters using validation
best_pipeline, all_results = pipeline.validate_parameters(
    train_data=train,
    validation_weeks=12, # Use last 12 weeks for validation
    param_list=[(1,1,1), (2,1,1), (1,1,2), (2,1,2), (1,0,1), (2,0,2), (12, 0, 2)]
)

=== PARAMETER VALIDATION ===
Testing 7 parameter combinations
Validation weeks: 12

--- Testing ARIMA(1,1,1) (1/7) ---
=== FITTING HIERARCHICAL ARIMA PIPELINE ===


/tmp/ipython-input-70-1262855583.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result['IsHoliday'] = result['IsHoliday'].fillna(False)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting

Successfully trained 45 store models
✅ Pipeline fitted successfully!
=== MAKING PREDICTIONS ===
Validation pairs matched: 35541
✅ ARIMA(1,1,1) - WMAE: 4820.94
🏃 View run ARIMA(1,1,1) at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2/runs/8485cba9a5d5492083e37e283c5793dc
🧪 View experiment at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2

--- Testing ARIMA(2,1,1) (2/7) ---
=== FITTING HIERARCHICAL ARIMA PIPELINE ===


/tmp/ipython-input-70-1262855583.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result['IsHoliday'] = result['IsHoliday'].fillna(False)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using 

Successfully trained 45 store models
✅ Pipeline fitted successfully!
=== MAKING PREDICTIONS ===
Validation pairs matched: 35541
✅ ARIMA(2,1,1) - WMAE: 5003.39
🏃 View run ARIMA(2,1,1) at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2/runs/c3e858816a02433db7128cac4cbceed5
🧪 View experiment at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2

--- Testing ARIMA(1,1,2) (3/7) ---
=== FITTING HIERARCHICAL ARIMA PIPELINE ===


...

Successfully trained 45 store models
✅ Pipeline fitted successfully!
=== MAKING PREDICTIONS ===
Validation pairs matched: 35541
✅ ARIMA(1,0,1) - WMAE: 5283.86
🏃 View run ARIMA(1,0,1) at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2/runs/7800778e99bc467ab119c856a6edb845
🧪 View experiment at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2

--- Testing ARIMA(2,0,2) (6/7) ---
=== FITTING HIERARCHICAL ARIMA PIPELINE ===


/tmp/ipython-input-70-1262855583.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result['IsHoliday'] = result['IsHoliday'].fillna(False)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using 

Successfully trained 45 store models
✅ Pipeline fitted successfully!
=== MAKING PREDICTIONS ===
Validation pairs matched: 35541
✅ ARIMA(2,0,2) - WMAE: 5336.38
🏃 View run ARIMA(2,0,2) at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2/runs/577eef2f94cf4931a18fc6f20916df71
🧪 View experiment at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2

--- Testing ARIMA(12,0,2) (7/7) ---
=== FITTING HIERARCHICAL ARIMA PIPELINE ===


/tmp/ipython-input-70-1262855583.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result['IsHoliday'] = result['IsHoliday'].fillna(False)
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.11/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likeliho

Successfully trained 45 store models
✅ Pipeline fitted successfully!
=== MAKING PREDICTIONS ===
Validation pairs matched: 35541
✅ ARIMA(12,0,2) - WMAE: 5501.35
🏃 View run ARIMA(12,0,2) at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2/runs/cf5b5dca24864078a2b55182f6035843
🧪 View experiment at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2
Best parameters (by WMAE):
1. ARIMA(1,1,1) - WMAE: 4820.94
2. ARIMA(2,1,1) - WMAE: 5003.39
3. ARIMA(1,1,2) - WMAE: 5004.89

✅ Best parameters: ARIMA(1, 1, 1) with WMAE: 4820.94
🏃 View run arima_parameter_validation at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2/runs/787ebe19eb5844609adb1cfe7db1b886
🧪 View experiment at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/2


In [73]:
# Train final model on full data with best parameters
if best_pipeline:
    import mlflow
    import pickle  # Add this import

    # Clean up any existing runs
    try:
        mlflow.end_run()
    except:
        pass

    # Start MLflow run for final model
    mlflow.set_experiment("final_best_model")

    with mlflow.start_run(run_name="best_arima_final_model"):
        # Log best parameters
        mlflow.log_param("best_p", best_pipeline.arima_trainer.p)
        mlflow.log_param("best_d", best_pipeline.arima_trainer.d)
        mlflow.log_param("best_q", best_pipeline.arima_trainer.q)
        mlflow.log_param("prediction_periods", 39)
        mlflow.log_param("trained_on_full_data", True)

        # Create and train final pipeline
        final_pipeline = HierarchicalARIMAPipeline(
            prediction_periods=39,
            p=best_pipeline.arima_trainer.p,
            d=best_pipeline.arima_trainer.d,
            q=best_pipeline.arima_trainer.q
        )

        final_pipeline.fit(train)

        # Log the final trained model - UPDATED
        model_filename = "best_arima_final_model.pkl"
        with open(model_filename, 'wb') as f:
            pickle.dump(final_pipeline, f)

        mlflow.log_artifact(model_filename, artifact_path="models")
        mlflow.log_param("model_filename", model_filename)
        mlflow.log_param("model_type", "HierarchicalARIMAPipeline")

        # Log training data info
        mlflow.log_param("training_data_size", len(train))
        mlflow.log_param("training_date_range", f"{train['Date'].min()} to {train['Date'].max()}")
        mlflow.log_metric("num_successful_store_models", len(final_pipeline.arima_trainer.store_models))

        # Generate final submission
        submission = final_pipeline.predict_for_test(
            test_data=test,
            save_path='/content/drive/MyDrive/ML_Final_Project/best_hierarchical_arima.csv'
        )

        # Log submission info
        mlflow.log_metric("submission_rows", len(submission))
        mlflow.log_artifact('/content/drive/MyDrive/ML_Final_Project/best_hierarchical_arima.csv')

=== FITTING HIERARCHICAL ARIMA PIPELINE ===


/tmp/ipython-input-70-1262855583.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result['IsHoliday'] = result['IsHoliday'].fillna(False)
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.11/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting

Successfully trained 45 store models
✅ Pipeline fitted successfully!
=== MAKING TEST PREDICTIONS ===
Test data shape: (115064, 4)
=== MAKING PREDICTIONS ===
Final submission shape: (115064, 2)
Required shape: (115064, 2)
Shape matches: True
Missing predictions filled: 0
✅ Submission saved to: /content/drive/MyDrive/ML_Final_Project/best_hierarchical_arima.csv
🏃 View run best_arima_final_model at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/3/runs/f0940165dac247de865d3503068d64bd
🧪 View experiment at: https://dagshub.com/dimna21/ML_Final_Project.mlflow/#/experiments/3


#REDO

AR I MA

AR coefficients are for timestamps picked using PACF.
I is for difference degree between 2 timestamps.
MA coefficients are for timestamps picked using ACF.

In [6]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/train.csv')
test = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/test.csv')
features = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/features.csv')
stores = pd.read_csv('/content/drive/MyDrive/ML_Final_Project/stores.csv')

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class BaseMerger(BaseEstimator, TransformerMixin):

    def __init__(self, features, stores):
        self.feature_store = features.merge(stores, how='inner', on='Store')
        self.feature_store['Date'] = pd.to_datetime(self.feature_store['Date'])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['Date'] = pd.to_datetime(X['Date'])
        merged = X.merge(self.feature_store, how='inner', on=['Store', 'Date', 'IsHoliday'])
        merged = merged.sort_values(by=['Date', 'Store', 'Dept']).reset_index(drop=True)
        return merged


In [10]:
merger = BaseMerger(features, stores)
x_merged = merger.fit_transform(train)

In [12]:
class FeatureAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.superbowl = pd.to_datetime(['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08'])
        self.labor_day = pd.to_datetime(['2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06'])
        self.thanksgiving = pd.to_datetime(['2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29'])
        self.christmas = pd.to_datetime(['2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27'])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Convert temperature to Celsius
        if 'Temperature' in X.columns:
            X['Temperature'] = (X['Temperature'] - 32) * (5.0 / 9.0)

        # Basic date parts
        X['Day'] = X['Date'].dt.day
        X['Month'] = X['Date'].dt.month
        X['Year'] = X['Date'].dt.year

        # Extract ISO week and year for holiday matching
        X['Week'] = X['Date'].dt.isocalendar().week
        X['YearNum'] = X['Date'].dt.year

        # Helper to flag if a date is in same ISO week/year as a known holiday
        def is_holiday_week(date_series, holidays):
            holiday_weeks = set((d.isocalendar().week, d.year) for d in holidays)
            return date_series.apply(lambda d: (d.isocalendar().week, d.year) in holiday_weeks if pd.notnull(d) else False).astype(int)

        X['SuperbowlWeek'] = is_holiday_week(X['Date'], self.superbowl)
        X['LaborDayWeek'] = is_holiday_week(X['Date'], self.labor_day)
        X['ThanksgivingWeek'] = is_holiday_week(X['Date'], self.thanksgiving)
        X['ChristmasWeek'] = is_holiday_week(X['Date'], self.christmas)

        # Calculate days to Thanksgiving and Christmas (using Nov 24 and Dec 24 as anchor dates)
        thanksgiving_dates = pd.to_datetime(X['Year'].astype(str) + "-11-24")
        christmas_dates = pd.to_datetime(X['Year'].astype(str) + "-12-24")

        X['Days_to_Thanksgiving'] = (thanksgiving_dates - X['Date']).dt.days
        X['Days_to_Christmas'] = (christmas_dates - X['Date']).dt.days

        # Clean up helper cols
        X = X.drop(columns=['Week', 'YearNum'])

        return X

In [14]:
feature_adder = FeatureAdder()
x_features = feature_adder.fit_transform(x_merged)

In [20]:
class MissingValueFiller(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.markdown_cols = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        self.mean_cols = ['CPI', 'Unemployment']
        self.mean_values = {}

    def fit(self, X, y=None):
        for col in self.mean_cols:
            if col in X.columns:
                self.mean_values[col] = X[col].mean()
        return self

    def transform(self, X):
        X = X.copy()

        # Fill markdowns with 0
        for col in self.markdown_cols:
            if col in X.columns:
                X[col] = X[col].fillna(0.0)

        # Fill CPI and Unemployment with learned mean
        for col in self.mean_cols:
            if col in X.columns and col in self.mean_values:
                X[col] = X[col].fillna(self.mean_values[col])

        return X

In [24]:
missing_filler = MissingValueFiller()
x_filled = missing_filler.fit_transform(x_features)

In [25]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.type_mapping = {'A': 3, 'B': 2, 'C': 1}
        self.holiday_mapping = {False: 0, True: 1}

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        if 'Type' in X.columns:
            X['Type'] = X['Type'].map(self.type_mapping)

        if 'IsHoliday' in X.columns:
            X['IsHoliday'] = X['IsHoliday'].map(self.holiday_mapping)

        return X

In [26]:
cat_encoder = CategoricalEncoder()
x_encoded = cat_encoder.fit_transform(x_filled)

In [55]:
class StoreAggregator(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.timeseries = {}

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        self.timeseries = {}

        for store in X['Store'].unique():
            self.aggregate_store_info(store, X)

        return self.timeseries

    def aggregate_store_info(self, store_id, X):

        store_data = X[X['Store'] == store_id].copy()
        sum_columns = ['Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
        first_columns = ['IsHoliday', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
                        'Type', 'Size', 'Day', 'Month', 'Year', 'SuperbowlWeek',
                        'LaborDayWeek', 'ThanksgivingWeek', 'ChristmasWeek',
                        'Days_to_Thanksgiving', 'Days_to_Christmas']
        agg_dict = {}

        for col in sum_columns:
            if col in store_data.columns:
                agg_dict[col] = 'sum'

        for col in first_columns:
            if col in store_data.columns:
                agg_dict[col] = 'first'

        aggregated = store_data.groupby(['Date', 'Store']).agg(agg_dict).reset_index()
        aggregated = aggregated.sort_values('Date').reset_index(drop=True)
        dept_proportions = self.calculate_dept_proportions(store_data)

        self.timeseries[store_id] = (aggregated, dept_proportions)

        return aggregated

    def calculate_dept_proportions(self, store_data):

      dept_totals = store_data.groupby('Dept')['Weekly_Sales'].sum()
      store_total = store_data['Weekly_Sales'].sum()

      if store_total == 0:
          num_depts = len(dept_totals)
          return {dept: 1.0/num_depts for dept in dept_totals.index}

      dept_proportions = (dept_totals / store_total).to_dict()
      return dept_proportions


In [56]:
store_aggregator = StoreAggregator()
x_series = store_aggregator.fit_transform(x_encoded)

In [77]:
df, prop = x_series[1]

In [79]:
df.columns

Index(['Date', 'Store', 'Weekly_Sales', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'IsHoliday', 'Temperature', 'Fuel_Price',
       'CPI', 'Unemployment', 'Type', 'Size', 'Day', 'Month', 'Year',
       'SuperbowlWeek', 'LaborDayWeek', 'ThanksgivingWeek', 'ChristmasWeek',
       'Days_to_Thanksgiving', 'Days_to_Christmas'],
      dtype='object')

In [106]:
from darts import TimeSeries
from darts.models.forecasting.arima import ARIMA

ts = TimeSeries.from_dataframe(
    df,
    time_col='Date',
    value_cols=['Weekly_Sales']
)

covariate_columns = [col for col in df.columns if col not in ['Date', 'Weekly_Sales', 'Store']]
future_covariates = TimeSeries.from_dataframe(
    df,
    time_col='Date',
    value_cols=covariate_columns
)


In [111]:
import numpy as np

train_ts = ts[:130]  # First 130 weeks for training
val_ts = ts[130:]    # Remaining weeks for validation

model = ARIMA(p=1, d=1, q=1)
model.fit(train_ts, future_covariates=future_covariates)
predictions = model.predict(n=len(val_ts), future_covariates=future_covariates)


Validation period: 13 weeks
WMAE: 93673.36

First 5 weeks comparison:
Week 1: Actual=1,631,136, Predicted=1,626,045, Error=5,091
Week 2: Actual=1,592,410, Predicted=1,560,287, Error=32,123
Week 3: Actual=1,597,868, Predicted=1,523,139, Error=74,729
Week 4: Actual=1,494,122, Predicted=1,458,112, Error=36,010
Week 5: Actual=1,582,083, Predicted=1,507,294, Error=74,790
